In [9]:
import os
import numpy as np
from skimage.color import rgb2gray
from skimage.feature import hog
from PIL import Image, UnidentifiedImageError

In [10]:
def img_to_np(img_path):
    try:
        img = Image.open(img_path)
        img = img.convert("RGB") # ensure img is in RGB
        new_img = img.resize((224, 224))
        return np.array(new_img)
    
    except (IOError, UnidentifiedImageError):
        return None

In [11]:
def row_of_features(img):
    # 3 channel img to single row
    color_features = img.flatten()

    gray_img = rgb2gray(img)

    hog_features = hog(gray_img, block_norm='L2-Hys', pixels_per_cell=(16, 16))

    # combine color & hog features into a single array
    row = np.hstack((color_features, hog_features))

    return row

In [12]:
def create_feature_matrix(folder_path):
    list_of_arr = []
    value = 0

    if folder_path.endswith("Bombus"):
        value = 1

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

        np_img = img_to_np(img_path)

        if np_img is None:
            continue

        row = row_of_features(np_img)

        row = np.hstack((row, value)) # append label at the end

        list_of_arr.append(row)

    matrix = np.array(list_of_arr)

    return matrix

In [13]:
mat1 = create_feature_matrix("Apis")
mat2 = create_feature_matrix("Bombus")

feature_matrix = np.vstack((mat1, mat2))

In [ ]:
np.random.shuffle(feature_matrix) # randomly shuffle rows of matrix
np.save("feature_matrix.npy", feature_matrix)